In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import os.path
import torch
import torch.utils.data as data
import skimage.io as io
from skimage import color
import cv2
from google.colab import drive
from torchvision import datasets, models, transforms
import copy
from torch.optim import lr_scheduler

### only for reference
import time
import torch
import random
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
# Leave this
num_epochs = 10

# Tune these
learning_rate = 3e-1
batch_size = 16 # Note: anything above 16 might cause cuda to run out of memory

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os

GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', 'EECS 545 Project')
print(os.listdir(GOOGLE_DRIVE_PATH))

os.chdir(GOOGLE_DRIVE_PATH)
print(os.getcwd())

['metadata.json', 'Image.zip', '545 Project Ideas.gdoc', 'data_scene_flow', 'Copy of Data_Loading.ipynb', '.ipynb_checkpoints', 'Image.zip (Unzipped Files)', 'Images', 'Untitled Diagram.drawio', 'Weather', 'Extra_Weather_Data', 'More_Weather_Data_60k', 'DataSplit.ipynb', 'Unet + triplet loss.ipynb', 'Data_Loading.ipynb', 'saved_model', 'Untitled', '__pycache__', 'Joey Unet and Content Loss.ipynb', 'Test version(Yuanbin)-UNet Triplet loss.ipynb', 'JoeyVAE598.py', 'ExampleInterpolation.png', 'newest Unet and Triplet Loss.ipynb', 'WeatherVAE.pt', 'Tune parameters.ipynb', 'Baselines.ipynb', 'WeatherVAE.ipynb', 'Performance', 'WeatherVAE-test (Jason).ipynb', 'JoeyTransferLearning.ipynb']
/content/drive/.shortcut-targets-by-id/1YO9ukO9embmobkuEcbxl71Qpmtma6P0Q/EECS 545 Project


In [5]:
### U-Net
def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )   


class UNet(nn.Module):

    def __init__(self, n_class):
        super().__init__()
                
        self.conv_down_1 = conv_block(3, 64)
        self.conv_down_2 = conv_block(64, 128)
        self.conv_down_3 = conv_block(128, 256)
        self.conv_down_4 = conv_block(256, 512)        

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)        
        
        self.conv_up_3 = conv_block(256 + 512, 256)
        self.conv_up_2 = conv_block(128 + 256, 128)
        self.conv_up_1 = conv_block(128 + 64, 64)
        
        self.conv_last = nn.Conv2d(64, n_class, 1)
        
        
    def forward(self, x):
        conv1 = self.conv_down_1(x)
        x = self.maxpool(conv1)

        conv2 = self.conv_down_2(x)
        x = self.maxpool(conv2)
        
        conv3 = self.conv_down_3(x)
        x = self.maxpool(conv3)   
        
        x = self.conv_down_4(x)
        
        x = self.upsample(x)        
        x = torch.cat([x, conv3], dim=1)
        
        x = self.conv_up_3(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv2], dim=1)       

        x = self.conv_up_2(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv1], dim=1)   
        
        x = self.conv_up_1(x)
        
        out = self.conv_last(x)

        return out

In [6]:
model = UNet(3)
model.load_state_dict(torch.load('/content/drive/MyDrive/EECS 545 Project/saved_model/model.pkl'))  ### remember to change the path
model.to("cuda")

UNet(
  (conv_down_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (conv_down_2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (conv_down_3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (conv_down_4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kerne

In [7]:
print(model)
encoder = nn.Sequential(
          model.conv_down_1,
          model.conv_down_2,
          model.conv_down_3,
          model.conv_down_4,
          model.maxpool
)
encoder.requires_grad = False
# decoder = nn.Sequential(
#     nn.Conv2d(512, 64, 3)
# )
encoder.to("cuda")
print(encoder)

UNet(
  (conv_down_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (conv_down_2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (conv_down_3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (conv_down_4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
  )
  (maxpool): MaxPool2d(kerne

In [8]:
class SmallDataLoader:
  def __init__(self, BatchSize=16, TestPercent=0.1, data_loc="Weather", img_size=224, dev="cuda"):
    self.train_names = []
    self.train_labels = []
    self.val_names = []
    self.val_labels = []
    self.test_names = []
    self.test_labels = []

    self.label_keys = {0: "cloudy", 1: "foggy", 2: "rain", 3: "snow", 4: "sunny"}
    self.data_loc = os.path.join(os.getcwd(), data_loc)
    self.batch_size = BatchSize
    self.img_size = img_size
    self.dev=dev

    data_types = ["cloudy", "foggy", "rain", "snow", "sunny"]
    splits = ["Train", "Val", "Test"]
    for split in splits:
      for i in range(5):
        dtype = self.label_keys[i]
        type_loc = os.path.join(data_loc, split, dtype)
        files = os.listdir(type_loc)

        if split == "Train":
          self.train_labels += len(files) * [i]
          self.train_names += files
        elif split == "Val":
          self.val_labels += len(files) * [i]
          self.val_names += files
        elif split == "Test":
          self.test_labels += len(files) * [i]
          self.test_names += files

    # Shuffle data first
    self.N_train = len(self.train_names) - len(self.train_names) % self.batch_size
    self.N_val = len(self.val_names) - len(self.val_names) % self.batch_size
    self.N_test = len(self.test_names) - len(self.test_names) % self.batch_size
    self.shuffle_train()

    self.iter_no = 0
    self.val_no = 0

    self.transform_dict = {
        "Train": transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.2),
        transforms.GaussianBlur(5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ]),
        "Val": transforms.Compose([
        transforms.CenterCrop(img_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
    }

  # index into file names, labels
  def get_ind(self, index, dset="Train"):
    if dset == "Train":
      fname = self.train_names[index]
      label = self.train_labels[index]
    elif dset == "Val":
      fname = self.val_names[index]
      label = self.val_labels[index]

    label_name = self.label_keys[label]
    img_loc = os.path.join(self.data_loc, dset, label_name, fname)
    input_image = Image.open(img_loc).convert('RGB')
    input_tensor = self.transform_dict[dset](input_image)

    return input_tensor, label

  # For training
  def get_batch(self):
    # End of Epoch
    if self.iter_no + self.batch_size >= self.N_train:
      self.iter_no = 0
      return "EOE", None

    batch_img = torch.zeros((self.batch_size, 3, self.img_size, self.img_size), device=self.dev)
    batch_lab = torch.zeros((self.batch_size), device=self.dev, dtype=torch.long)
    for i in range(self.batch_size):
      input_tensor, label = self.get_ind(self.iter_no + i)
      batch_img[i] = input_tensor
      batch_lab[i] = label
    self.iter_no += self.batch_size
    return batch_img, batch_lab

  def get_val(self):
    if self.val_no + self.batch_size >= self.N_val:
      self.val_no = 0
      return "EOE", None

    batch_img = torch.zeros((self.batch_size, 3, self.img_size, self.img_size), device=self.dev)
    batch_lab = torch.zeros((self.batch_size), device=self.dev, dtype=torch.long)
    for i in range(self.batch_size):
      input_tensor, label = self.get_ind(self.val_no + i, dset="Val")
      batch_img[i] = input_tensor
      batch_lab[i] = label
    self.val_no += self.batch_size
    return batch_img, batch_lab


  def shuffle_train(self):
    inds = np.random.choice(self.N_train, size=self.N_train, replace=False)
    self.train_names = [self.train_names[i] for i in inds]
    self.train_labels = [self.train_labels[i] for i in inds]
    self.iter_no = 0

In [9]:
dl = SmallDataLoader(BatchSize=batch_size)

decoder = nn.Sequential(
    # Decreasing number of filters b/c too big for linear layers rn
    nn.Conv2d(512, 64, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(5, stride=2, padding=2),
    nn.Conv2d(64, 8, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(5, stride=2, padding=2),
    nn.Conv2d(8, 2, 3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(5, stride=2, padding=2),
    # Now using linear layers
    nn.Flatten(),
    nn.Linear(2*14*14, 100),
    nn.Linear(100, 5)
)

decoder.requires_grad = True

# Test
with torch.no_grad():
  imgs, labels = dl.get_batch()
  encodings = encoder(imgs)
  decoder.to("cuda")
  print(decoder(encodings).shape)
  print(encodings.shape)

torch.Size([16, 5])
torch.Size([16, 512, 112, 112])


In [10]:
weather_model = nn.Sequential(
    encoder,
    decoder
)

# The model
print(weather_model)
# Making sure it is frozen
print(weather_model[0].requires_grad)
print(weather_model[1].requires_grad)

Sequential(
  (0): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
    (2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
    (3): Sequential(
      (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
 

In [11]:
# Now try training it!
def train_model(model, criterion, optimizer, scheduler, data_loader, num_epochs=10):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    train_hist = []
    val_hist = []

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            data_loader.shuffle_train()
            while True:
                if phase == "train":
                  inputs, labels = data_loader.get_batch()
                else:
                  inputs, labels = data_loader.get_val()
                # End of epoch
                if inputs == "EOE":
                  break

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
            if phase == "train":
              epoch_loss = running_loss / data_loader.N_train
              epoch_acc = running_corrects / data_loader.N_train
            else:
              epoch_loss = running_loss / data_loader.N_val
              epoch_acc = running_corrects / data_loader.N_val

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            if phase == "train":
              train_hist.append(epoch_acc)
            else:
              val_hist.append(epoch_acc)

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, train_hist, val_hist

In [ ]:
criterion = nn.CrossEntropyLoss()

# Only the decoder is being optimized
optimizer_ft = optim.Adam(weather_model[1].parameters(), lr=learning_rate)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.1)

model, train_hist, val_hist = train_model(weather_model, criterion, optimizer_ft, exp_lr_scheduler, dl, num_epochs=num_epochs)

Epoch 0/9
----------
train Loss: 7507355.0743 Acc: 0.2146
val Loss: 5.9536 Acc: 0.1953
Epoch 1/9
----------
train Loss: 2.0287 Acc: 0.2271
val Loss: 1.8644 Acc: 0.2227
Epoch 2/9
----------
train Loss: 1.5788 Acc: 0.2118
val Loss: 1.5738 Acc: 0.1953
Epoch 3/9
----------
